In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Dataset Path
DATASET_PATH = r"C:\Users\user\Autolighting\UTKFace"  # Folder where files will be extracted


# Load images and labels
images = []
labels = []

for file in os.listdir(DATASET_PATH):
    try:
        if file.endswith(".jpg"):
            # Filename format: age_gender_race.jpg (e.g., 25_1_2.jpg -> Gender: 1 (Male), 0 (Female))
            parts = file.split("_")
            gender = int(parts[1])  # Gender label (0 = Female, 1 = Male)

            img_path = os.path.join(DATASET_PATH, file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))  # Resize image
            img = img / 255.0  # Normalize pixel values

            images.append(img)
            labels.append(gender)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Convert to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification (Male/Female)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save the trained model
model.save("gender_model.h5")


In [ ]:
import cv2
import numpy as np
import time
import argparse
from tensorflow.keras.models import load_model

# Load the pre-trained deep learning model
MODEL_PATH = "gender_model.h5"  # Replace with your model's path
model = load_model(MODEL_PATH)

# Define gender categories
gender_classes = ['Male', 'Female']

# Load OpenCV's pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Initialize Raspberry Pi Camera
camera = cv2.VideoCapture(0)

def preprocess_face(face):
    """Preprocess face for gender classification."""
    face = cv2.resize(face, (64, 64))  # Resize to model input size
    face = face / 255.0  # Normalize pixel values
    face = np.expand_dims(face, axis=0)  # Add batch dimension
    return face

while True:
    ret, frame = camera.read()
    if not ret:
        continue
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        processed_face = preprocess_face(face)
        
        # Predict gender
        prediction = model.predict(processed_face)[0]
        gender = gender_classes[np.argmax(prediction)]
        
        # Display result
        label = f"Gender: {gender}"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    cv2.imshow("Gender Classification", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()
